In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import numpy as np
import os
import json



In [2]:
df = pd.read_json('./data/data.json', orient='records')
df.head()

,name,img,cmc,colours,type,subtype,power,toughness,loyalty,tokens
0,+Two Mace,https://cards.scryfall.io/normal/front/e/8/e88...,0.1250,"[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.00,0.00,0.0,"[1, 96, 0, 40, 186, 96, 116, 116, 10, 169, 12,..."
1,Aarakocra Sneak,https://cards.scryfall.io/normal/front/2/a/2a8...,0.2500,"[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.05,0.20,0.0,"[1, 50, 30, 3, 0, 23, 4, 428, 9, 629, 2]"
2,"Aatchik, Emerald Radian",https://cards.scryfall.io/normal/front/f/b/fbd...,0.3750,"[0, 0, 1, 0, 1]","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0,...",0.15,0.15,0.0,"[1, 30, 52, 23, 65, 5, 131, 162, 430, 0, 44, 4..."
3,Abaddon the Despoiler,https://cards.scryfall.io/normal/front/c/9/c9f...,0.3125,"[0, 1, 1, 1, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.25,0.25,0.0,"[1, 128, 152, 11, 15, 31, 4, 29, 34, 11, 53, 2..."
4,Abandoned Campground,https://cards.scryfall.io/normal/front/e/e/ee0...,0.0000,"[1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.00,0.00,0.0,"[1, 3, 60, 23, 74, 159, 5, 37, 88, 677, 16, 12..."


In [3]:
class MTGCardDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        # Get a single row from the dataframe
        row = self.dataframe.iloc[idx]
        
        # Extract token indices (for the RNN)
        original_tokens = row['tokens']
        shifted_tokens = [t + 1 for t in original_tokens]  # Add 1 to all indices Allows for padding
        tokens = torch.tensor(shifted_tokens, dtype=torch.long)
        
        # Extract numerical features (for the FC network)
        numerical_features = []
        
        # Add CMC (converted mana cost)
        numerical_features.append(row['cmc'])
        
        # Add other numerical features
        numerical_features.extend(row['colours'])
        numerical_features.extend(row['type'])
        numerical_features.extend(row['subtype'])
        numerical_features.append(row['power'])
        numerical_features.append(row['toughness'])
        numerical_features.append(row['loyalty'])
        
        # Convert to tensor
        numerical_features = torch.tensor(numerical_features, dtype=torch.float)
        
        return {
            'tokens': tokens,
            'numerical_features': numerical_features,
            'name': row['name'],
            'img': row['img']
        }

dataset = MTGCardDataset(df)  # df is your pandas dataframe

In [4]:
dataset[0]

{'tokens': tensor([  2,  97,   1,  41, 187,  97, 117, 117,  11, 170,  13,  14,   1,   5,
          15,  97, 100,  34,   6, 109,   3]),
 'numerical_features': tensor([ 0.1250,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000]),
 'name': '+Two Mace',
 'img': 'https://cards.scryfall.io/normal/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221'}

In [5]:
def collate_mtg_cards(batch):
    """
    Custom collate function to handle variable-length token sequences.
    """
    # Extract each element from the batch
    names = [item['name'] for item in batch]
    imgs = [item['img'] for item in batch]
    token_sequences = [item['tokens'] for item in batch]
    numerical_features = [item['numerical_features'] for item in batch]
    
    # Get lengths of each sequence for packing
    lengths = torch.tensor([len(seq) for seq in token_sequences], dtype=torch.long)
    
    # Sort sequences by length in descending order for efficient packing
    sorted_indices = torch.argsort(lengths, descending=True)
    sorted_lengths = lengths[sorted_indices]
    sorted_token_sequences = [token_sequences[i] for i in sorted_indices]
    sorted_numerical_features = [numerical_features[i] for i in sorted_indices]
    sorted_names = [names[i] for i in sorted_indices]
    sorted_imgs = [imgs[i] for i in sorted_indices]
    
    # Pad token sequences
    padded_tokens = pad_sequence(sorted_token_sequences, batch_first=True, padding_value=0)
    
    # Stack numerical features
    stacked_numerical_features = torch.stack(sorted_numerical_features)
    
    return {
        'tokens': padded_tokens,
        'token_lengths': sorted_lengths,
        'numerical_features': stacked_numerical_features,
        'names': sorted_names,
        'imgs': sorted_imgs
    }




In [6]:
# Assuming df is your DataFrame
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Now create your datasets
train_dataset = MTGCardDataset(train_df)
val_dataset = MTGCardDataset(val_df)

# Create your dataloaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_mtg_cards
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=collate_mtg_cards
)

In [7]:
def load_word2vec_embedding(model_path, padding_idx=0):
    word2vec_model = Word2Vec.load(model_path)
    
    # Get embedding dimension from the model
    embedding_dim = word2vec_model.vector_size
    vocab_size = len(word2vec_model.wv)
    
    # Initialize embedding matrix with zeros
    # Add 1 to vocab_size to account for padding token
    embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))
    
    # Fill the embedding matrix with word vectors
    for i, word in enumerate(word2vec_model.wv.index_to_key):
        # Add 1 to index to reserve index 0 for padding
        embedding_matrix[i + 1] = word2vec_model.wv[word]
    
    # Convert to torch tensor
    embedding_weights = torch.FloatTensor(embedding_matrix)
    
    # Create embedding layer initialized with pre-trained weights
    embedding_layer = nn.Embedding.from_pretrained(
        embedding_weights,
        padding_idx=padding_idx,
        freeze=False  # Set to True if you don't want to fine-tune the embeddings
    )
    
    return embedding_layer, vocab_size + 1  


In [8]:
class MTGCardAutoencoderWithTextDecoding(nn.Module):
    def __init__(self, vocab_size, embedding_layer, hidden_dim, numerical_dim, latent_dim):
        super(MTGCardAutoencoderWithTextDecoding, self).__init__()
        
        # Use pre-trained embedding layer
        self.embedding = embedding_layer
        embedding_dim = embedding_layer.embedding_dim
        
        # --- ENCODER COMPONENTS ---
        # RNN for encoding token sequences
        self.encoder_rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        
        # FC network for processing numerical features
        self.fc_numerical = nn.Sequential(
            nn.Linear(numerical_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Combined encoder to latent space
        self.fc_combined = nn.Linear(hidden_dim * 2, latent_dim)
        
        # --- DECODER COMPONENTS ---
        # Decoder for numerical features
        self.numerical_decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, numerical_dim),
            nn.Tanh()  # For normalized numerical features
        )
        
        # Decoder for token sequences
        self.decoder_initial = nn.Linear(latent_dim, hidden_dim)
        self.decoder_rnn = nn.GRU(embedding_dim + latent_dim, hidden_dim, batch_first=True)
        self.decoder_fc = nn.Linear(hidden_dim, vocab_size)
        
        # Store for later use
        self.vocab_size = vocab_size
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        
    def encode(self, tokens, token_lengths, numerical_features):
        # Process token sequences with RNN
        embedded = self.embedding(tokens)
        
        # Pack padded sequence
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, token_lengths, batch_first=True
        )
        
        _, rnn_hidden = self.encoder_rnn(packed)
        rnn_hidden = rnn_hidden.squeeze(0)  # Shape: [batch_size, hidden_dim]
        
        # Process numerical features
        numerical_encoding = self.fc_numerical(numerical_features)
        
        # Combine both encodings
        combined = torch.cat([rnn_hidden, numerical_encoding], dim=1)
        latent = self.fc_combined(combined)
        
        return latent
    
    def decode_numerical(self, latent):
        # Decode numerical features
        return self.numerical_decoder(latent)
    
    def decode_tokens(self, latent, tokens, teacher_forcing_ratio=0.5):
        batch_size = latent.size(0)
        max_len = tokens.size(1)
        
        # Initialize tensor to store outputs
        outputs = torch.zeros(batch_size, max_len, self.vocab_size).to(latent.device)
        
        # Initialize decoder hidden state from latent vector
        hidden = self.decoder_initial(latent).unsqueeze(0)  # Add sequence dimension
        
        # First input is the special start token (we'll use zeros for simplicity)
        decoder_input = torch.zeros(batch_size, 1, self.embedding.embedding_dim).to(latent.device)
        
        # Expand latent vector to be concatenated with each input
        expanded_latent = latent.unsqueeze(1).expand(-1, max_len, -1)
        
        for t in range(max_len):
            # Concatenate decoder input with latent vector for each time step
            decoder_input_with_latent = torch.cat([decoder_input, latent.unsqueeze(1)], dim=2)
            
            # Feed through RNN
            output, hidden = self.decoder_rnn(decoder_input_with_latent, hidden)
            
            # Project to vocabulary space
            output = self.decoder_fc(output.squeeze(1))
            outputs[:, t] = output
            
            # Teacher forcing: use ground truth as next input with probability
            use_teacher_forcing = (torch.rand(1).item() < teacher_forcing_ratio)
            
            if use_teacher_forcing and t < max_len - 1:
                # Use actual next token from target
                decoder_input = self.embedding(tokens[:, t+1].unsqueeze(1))
            else:
                # Use own prediction
                top1 = output.max(1)[1]
                decoder_input = self.embedding(top1.unsqueeze(1))
        
        return outputs
    
    def forward(self, tokens, token_lengths, numerical_features, teacher_forcing_ratio=0.5):
        # Encode input to latent space
        latent = self.encode(tokens, token_lengths, numerical_features)
        
        # Decode numerical features
        reconstructed_numerical = self.decode_numerical(latent)
        
        # Decode token sequence
        reconstructed_tokens = self.decode_tokens(latent, tokens, teacher_forcing_ratio)
        
        return reconstructed_numerical, reconstructed_tokens, latent



In [9]:
# Initialize the model
hidden_dim = 128
numerical_dim = len(dataset[0]['numerical_features'])
latent_dim = 64  # Desired embedding dimension

embedding_layer, vocab_size = load_word2vec_embedding("./models/word2vec.model")

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MTGCardAutoencoderWithTextDecoding(vocab_size, embedding_layer, hidden_dim, numerical_dim, latent_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Loss functions
numerical_criterion = nn.MSELoss()
token_criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding tokens (index 0)
alpha = 0.5  # Balance between numerical and token losses

## Evaluation

def evaluate(model, val_dataloader, numerical_criterion, token_criterion, device):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            # Extract inputs from batch
            tokens = batch['tokens'].to(device)
            token_lengths = batch['token_lengths']
            numerical_features = batch['numerical_features'].to(device)
            
            # Forward pass
            reconstructed_numerical, reconstructed_tokens, latent = model(
                tokens, token_lengths, numerical_features, teacher_forcing_ratio=0.5
            )
            
            # Compute numerical reconstruction loss
            numerical_loss = numerical_criterion(reconstructed_numerical, numerical_features)
            
            # Compute token reconstruction loss
            token_preds = reconstructed_tokens.view(-1, vocab_size)
            token_targets = tokens.view(-1)
            token_loss = token_criterion(token_preds, token_targets)
            
            # Combine losses
            loss = alpha * numerical_loss + (1 - alpha) * token_loss
            val_loss += loss.item()

    return val_loss / len(val_dataloader)


In [10]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    numerical_losses = 0
    token_losses = 0
    
    for batch in train_dataloader:
        tokens = batch['tokens'].to(device)
        token_lengths = batch['token_lengths']
        numerical_features = batch['numerical_features'].to(device)
        
        # Forward pass
        reconstructed_numerical, reconstructed_tokens, latent = model(
            tokens, token_lengths, numerical_features, teacher_forcing_ratio=0.5
        )
        
        # Compute numerical reconstruction loss
        numerical_loss = numerical_criterion(reconstructed_numerical, numerical_features)
        
        # Compute token reconstruction loss
        # Reshape predictions to [batch_size*seq_len, vocab_size]
        token_preds = reconstructed_tokens.view(-1, vocab_size)
        # Reshape targets to [batch_size*seq_len]
        token_targets = tokens.view(-1)
        token_loss = token_criterion(token_preds, token_targets)
        
        # Combine losses        
        loss = alpha * numerical_loss + (1 - alpha) * token_loss
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        numerical_losses += numerical_loss.item()
        token_losses += token_loss.item()
    
    val_loss = evaluate(model, val_dataloader, numerical_criterion, token_criterion, device)

    print(f"\nEpoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {total_loss/len(train_dataloader):.6f} - ("
          f"Numerical: {numerical_losses/len(train_dataloader):.6f}, "
          f"Token: {token_losses/len(train_dataloader):.6f}), "
          f" Val Loss:{val_loss:.6f}")



Epoch [1/50], Train Loss: 1.939921 - (Numerical: 0.030047, Token: 3.849796),  Val Loss:1.463849

Epoch [2/50], Train Loss: 1.215432 - (Numerical: 0.015697, Token: 2.415167),  Val Loss:1.078084

Epoch [3/50], Train Loss: 0.964046 - (Numerical: 0.012531, Token: 1.915561),  Val Loss:0.897568

Epoch [4/50], Train Loss: 0.862638 - (Numerical: 0.010938, Token: 1.714338),  Val Loss:0.831638

Epoch [5/50], Train Loss: 0.775891 - (Numerical: 0.009701, Token: 1.542081),  Val Loss:0.776536

Epoch [6/50], Train Loss: 0.735041 - (Numerical: 0.008856, Token: 1.461226),  Val Loss:0.752625

Epoch [7/50], Train Loss: 0.688282 - (Numerical: 0.008183, Token: 1.368382),  Val Loss:0.676089

Epoch [8/50], Train Loss: 0.659735 - (Numerical: 0.007654, Token: 1.311815),  Val Loss:0.656277

Epoch [9/50], Train Loss: 0.635300 - (Numerical: 0.007178, Token: 1.263422),  Val Loss:0.661609

Epoch [10/50], Train Loss: 0.614003 - (Numerical: 0.006686, Token: 1.221320),  Val Loss:0.637570

Epoch [11/50], Train Loss: 0

In [11]:
torch.save(model.state_dict(), "./models/AE_weights.pt")

In [10]:
state_dict = torch.load("./models/AE_weights.pt", map_location=torch.device('cuda'))
model.load_state_dict(state_dict)

C:\Users\Benwm\AppData\Local\Temp\ipykernel_17376\1658437441.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("./models/AE_weights.pt", map_locati

<All keys matched successfully>

In [12]:
# Generate embeddings for all cards
model.eval()
all_embeddings = []
all_names = []
all_imgs = []

dataset = MTGCardDataset(df)  # df is your pandas dataframe

dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_mtg_cards
)

with torch.no_grad():
    for batch in dataloader:
        tokens = batch['tokens'].to(device)
        token_lengths = batch['token_lengths']
        numerical_features = batch['numerical_features'].to(device)
        names = batch['names']
        imgs = batch['imgs']
        
        _, _, latent = model(tokens, token_lengths, numerical_features)
        
        all_embeddings.append(latent.cpu().numpy())
        all_names.extend(names)
        all_imgs.extend(imgs)

all_embeddings = np.vstack(all_embeddings)

In [13]:
from datetime import datetime

def save_embeddings(embeddings, names, imgs, directory="saved_embeddings"):
    """
    Save embeddings and corresponding card names using various methods
    
    Args:
        embeddings: NumPy array of embeddings
        names: List of card names
        directory: Directory to save files in
    """
    os.makedirs(directory, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d")
    filename = f"{directory}/mtg_embeddings_{timestamp}.json"
    
    data = {
        'embeddings': embeddings.tolist(),
        'names': names,
        'imgs': imgs,
        'metadata': {
            'date': timestamp,
            'embedding_dim': embeddings.shape[1],
            'num_cards': len(names)
        }
    }
    
    with open(filename, 'w') as f:
        json.dump(data, f)
    print(f"Saved embeddings and names to {filename}")
    return filename

save_path = save_embeddings(all_embeddings, all_names, all_imgs)


Saved embeddings and names to saved_embeddings/mtg_embeddings_20250808.json
